## Trabalho 1 - Big Data
### Mineração de Dados Complexos - Unicamp 2025

**Nomes**
- Vitor de Oliveira Fernandez Araujo
- Vitor Sancho Cardoso


**Instruções iniciais**

*   Abra os links dos dados:
    * https://tinyurl.com/bigdata-mcu
*   Clique em "Adicionar atalho ao Drive"


# Solução

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Configuração do ambiente

In [3]:
!pip install pyspark

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

from datetime import datetime

appName = 'Big Data'
master = 'local[*]'

spark = SparkSession.builder     \
    .master(master) \
    .appName(appName) \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

## Leitura de dados

In [5]:
# Usar esta entrada para testes
#input_data = spark.sparkContext.textFile('file:///content/drive/My Drive/mcu/mcu_subset.csv')

In [20]:
# Usar esta entrada para entrega final
input_data = spark.sparkContext.textFile('file:///content/drive/My Drive/mcu/mcu.csv')

In [6]:
input_data.take(10)

[';character;line;movie;year;words;Adam McKay;Anna Boden;Art Marcum;Ashley Edward Miller;Chris McKenna;Christopher Ford;Christopher Markus;Christopher Yost;Craig Kyle;Don Payne;Drew Pearce;Edgar Wright;Eric Pearson;Erik Sommers;Geneva Robertson-Dworet;Hawk Ostby;James Gunn;Joe Cornish;Joe Robert Cole;John Francis Daley;Jon Watts;Jonathan Goldstein;Joss Whedon;Justin Theroux;Mark Fergus;Matt Holloway;Paul Rudd;Ryan Coogler;Ryan Fleck;Shane Black;Stephen McFeely;Zack Stentz',
 '98;JAMES RHODES;Something’s...seriously wrong with you, man. ;Iron Man;2008;6;False;False;True;False;False;False;False;False;False;False;False;False;False;False;False;True;False;False;False;False;False;False;False;False;True;True;False;False;False;False;False;False',
 '198;GENERAL GABRIEL;Listen, son -- it’s been three months without a single indication that Stark is still alive.  We can’t keep risking assets, least of all you. ;Iron Man;2008;27;False;False;True;False;False;False;False;False;False;False;False;Fals

## Exemplo de uso do pipeline

In [6]:
from transformers import pipeline

# Baixar e configurar pipeline do modelo
sentiment = pipeline('sentiment-analysis')



No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [57]:

result = sentiment("I am Groot...")


In [58]:
result

[{'label': 'POSITIVE', 'score': 0.85554039478302}]

In [9]:
result[0]['label']

'POSITIVE'

## Solução

In [10]:
# Inclua outros personagens de sua escolha
characters = {'tony stark', 'steve rogers', 'thanos', 'bruce banner', 'rocket', 't\'challa', 'peter parker', 'peter quill', 'hank pym', 'groot'}

In [29]:
import re

# Modifique a solução para implementar a função Map
def line_sentiment(line):
    try:
      personagem = line[1].lower()
      polaridade = sentiment(line[2])[0]['label']
      if polaridade == 'POSITIVE':
          polaridade = 1
      else:
          polaridade = -1
      yield (personagem, (polaridade, 1))
    except:
      yield None


def resilient_split(line):
  try:
    return line.split(';')
  except:
    return None

def resilient_filter_characters(line):
  try:
    return line[1].lower() in characters
  except:
    return False

parsed_data = input_data.map(resilient_split)
filtered_characters = parsed_data.filter(resilient_filter_characters)

In [30]:
character_sentiment = filtered_characters.flatMap(line_sentiment)

In [31]:
character_sentiment.take(10)

[('tony stark', (-1, 1)),
 ('tony stark', (1, 1)),
 ('tony stark', (1, 1)),
 ('tony stark', (1, 1)),
 ('tony stark', (-1, 1)),
 ('tony stark', (1, 1)),
 ('tony stark', (-1, 1)),
 ('tony stark', (1, 1)),
 ('tony stark', (1, 1)),
 ('tony stark', (-1, 1))]

In [32]:
character_sentiment.count()

4639

In [33]:
# Implemente e aplique um método reduce para acumulação dos sentimentos dos personagens
sentiment_sums = character_sentiment.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

In [34]:
# Implemente e aplique um método para calculo do sentimento médio
result = sentiment_sums.mapValues(lambda x: x[0]/x[1])

# Resultado Final


Apresente o resultado final da sua análise completa.

In [36]:
result.sortBy(lambda x: x[1]).collect()

[('bruce banner', -0.29815303430079154),
 ('peter quill', -0.26811594202898553),
 ('rocket', -0.21658986175115208),
 ('steve rogers', -0.150460593654043),
 ('hank pym', -0.1095890410958904),
 ('thanos', -0.10091743119266056),
 ('tony stark', -0.087248322147651),
 ('peter parker', -0.016333938294010888),
 ("t'challa", 0.06976744186046512),
 ('groot', 0.9166666666666666)]

Percebemos que a média do sentimento, para a maioria dos personagens, tende ao campo negativo. Para personagens mais sarcásticos ou até "auto-depreciativos", vemos que a média tende mais para baixo, como é o caso dos 3 primeiros: Bruce Banner, Peter Quill e Rocket. Personagens tidos como mais positivos e "alegres", como o Peter Parker, acabam ficando com a média neutra, próxima de zero. Curiosamente, o personagem T'Challa, o Pantera Negra, tem uma média levemente positiva, que talvez seja explicada por seu comportamento mais formal e educado. Por fim, para efeitos de curiosidade, incluímos a análise do Groot, que ficou extremamente próxima de 1, já que a maioria das variações de "I am Groot" são avaliadas com sentimento positivo pelo modelo.